In [67]:
my_input = """set i 31
set a 1
mul p 17
jgz p p
mul a 2
add i -1
jgz i -2
add a -1
set i 127
set p 735
mul p 8505
mod p a
mul p 129749
add p 12345
mod p a
set b p
mod b 10000
snd b
add i -1
jgz i -9
jgz a 3
rcv b
jgz b -1
set f 0
set i 126
rcv a
rcv b
set p a
mul p -1
add p b
jgz p 4
snd a
set a b
jgz 1 3
snd b
set f 1
add i -1
jgz i -11
snd a
jgz f -16
jgz a -19
"""

In [68]:
test_input = """set a 1
add a 2
mul a a
mod a 5
snd a
set a 0
rcv a
jgz a -1
set a 1
jgz a -2"""

In [69]:
from collections import defaultdict

class SoundRecovered(Exception):
    pass

class Duet:
    def __init__(self, data):
        self._instructions = self._discover_instructions(data)
        self._registers = defaultdict(int)
        self._played_sound_value = None
        self._perform_instructions()
        
    @property
    def recovered_frequency(self):
        return self._played_sound_value
        
    def _discover_instructions(self, data):
        instructions_list = list()
        
        for instruction in data.splitlines():
            if instruction.startswith("snd"):
                action = [self._play_sound]
            elif instruction.startswith("set"):
                action = [self._set]
            elif instruction.startswith("add"):
                action = [self._increse]
            elif instruction.startswith("mul"):
                action = [self._multiply]
            elif instruction.startswith("mod"):
                action = [self._modulo]
            elif instruction.startswith("rcv"):
                action = [self._recover]
            elif instruction.startswith("jgz"):
                action = [self._jump]
            else:
                raise NotImplementedError()
                
            action += self._get_action_arguments(instruction)
            
            instructions_list.append(action)
            
        return instructions_list
    
    def _perform_instructions(self):
        i = 0
        while len(self._instructions) > i:
            action, *args = self._instructions[i]
            try:
                i += action(*args)
            except SoundRecovered:
                return
    
    def _play_sound(self, register):
        self._played_sound_value = self._registers[register]
        return 1
    
    def _set(self, register, value):
        self._registers[register] = value if isinstance(value, int) else self._registers[value]
        return 1
    
    def _increse(self, register, value):
        self._registers[register] += value if isinstance(value, int) else self._registers[value]
        return 1
    
    def _multiply(self, register, value):
        self._registers[register] *= value if isinstance(value, int) else self._registers[value]
        return 1
    
    def _modulo(self, register, value):
        self._registers[register] %= value if isinstance(value, int) else self._registers[value]
        return 1
    
    def _recover(self, register):
        if self._registers[register] != 0:
            raise SoundRecovered()
            
        return 1
    
    def _jump(self, register, value):
        register_value = register if isinstance(register, int) else self._registers[register]
        if register_value <= 0:
            return 1
        
        return value if isinstance(value, int) else self._registers[value]
    
    def _get_action_arguments(self, instruction):
        args = list()
        
        for argument in instruction[4:].split(" "):
            try:
                argument = int(argument)
            except ValueError:
                pass
            
            args.append(argument)
        
        return args

## Part 1

In [70]:
assert(Duet(test_input).recovered_frequency == 4)
print("Test passed")

Test passed


In [71]:
Duet(my_input).recovered_frequency

8600

## Part 2

In [72]:
from collections import defaultdict, deque

class EmptyBuffer(Exception):
    pass

class Duet:
    def __init__(self, data, program_id=0):
        self._send_queue = None
        self._reciving_queue = deque()
        self._instructions = self._discover_instructions(data)
        self._registers = defaultdict(lambda: program_id)
        self.send_counter = 0
        self._current_instruction_index = 0
        self._instructions_complete = False
    
    @property
    def terminated(self):
        return self._instructions_complete or self._is_buffer_empty()
    
    def connect_duet(self, duet):
        self._send_queue = duet._reciving_queue
        
    def run(self):   
        while len(self._instructions) > self._current_instruction_index:
            action, *args = self._instructions[self._current_instruction_index]
    
            try:
                self._current_instruction_index += action(*args)
            except EmptyBuffer:
                return
        
        self._instructions_complete = True
        
    def _discover_instructions(self, data):
        instructions_list = list()
        for instruction in data.splitlines():
            if instruction.startswith("snd"):
                action = [self._send]
            elif instruction.startswith("set"):
                action = [self._set]
            elif instruction.startswith("add"):
                action = [self._increse]
            elif instruction.startswith("mul"):
                action = [self._multiply]
            elif instruction.startswith("mod"):
                action = [self._modulo]
            elif instruction.startswith("rcv"):
                action = [self._recive]
            elif instruction.startswith("jgz"):
                action = [self._jump]
            else:
                raise NotImplementedError()
                
            action += self._get_action_arguments(instruction)
            instructions_list.append(action)
            
        return instructions_list
    
    def _send(self, register):
        value = register if isinstance(register, int) else self._registers[register]
        self._send_queue.append(value)
        self.send_counter += 1
        return 1
    
    def _set(self, register, value):
        self._registers[register] = value if isinstance(value, int) else self._registers[value]
        return 1
    
    def _increse(self, register, value):
        self._registers[register] += value if isinstance(value, int) else self._registers[value]
        return 1
    
    def _multiply(self, register, value):
        self._registers[register] *= value if isinstance(value, int) else self._registers[value]
        return 1
    
    def _modulo(self, register, value):
        self._registers[register] %= value if isinstance(value, int) else self._registers[value]
        return 1
    
    def _recive(self, register):
        if self._is_buffer_empty():
            raise EmptyBuffer()
            
        value = self._reciving_queue.popleft()
        self._registers[register] = value
        return 1
    
    def _jump(self, register, value):
        register_value = register if isinstance(register, int) else self._registers[register]
        if register_value <= 0:
            return 1
        
        return value if isinstance(value, int) else self._registers[value]
        
    def _is_buffer_empty(self):
        return not self._reciving_queue
    
    def _get_action_arguments(self, instruction):
        args = list()
        for argument in instruction[4:].split(" "):
            try:
                argument = int(argument)
            except ValueError:
                pass
            
            args.append(argument)
        
        return args

In [73]:
def execute_duets():
    duet_a = Duet(my_input, program_id=0)
    duet_b = Duet(my_input, program_id=1)
    duet_a.connect_duet(duet_b)
    duet_b.connect_duet(duet_a)

    while True:
        duet_a.run()
        duet_b.run()
        
        if duet_a.terminated and duet_b.terminated:
            break
    
    return duet_b.send_counter

In [74]:
print(execute_duets())

7239
